In [3]:
import os, argparse
import cv2, spacy, numpy as np
from keras.models import model_from_json
from keras.optimizers import SGD
import joblib
import sklearn.externals.joblib as joblib
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
from keras import backend as K
K.set_image_data_format('channels_first')
K.common.set_image_dim_ordering('th')
import warnings
from models.CNN.VGG import VGG_16

In [4]:
VQA_weights_file_name   = 'models/VQA/VQA_MODEL_WEIGHTS.hdf5'
label_encoder_file_name = 'models/VQA/FULL_labelencoder_trainval.pkl'
CNN_weights_file_name   = 'models/CNN/vgg16_weights.h5'

## Understanding VGG16

In [5]:
# load the model
model = VGG_16()
# summarize the model
model.summary()





Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_1 (ZeroPaddin (None, 3, 226, 226)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 224, 224)      1792      
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 64, 226, 226)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 224, 224)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 112, 112)      0         
_________________________________________________________________
zero_padding2d_3

In [6]:
def get_image_model(CNN_weights_file_name):
    from models.CNN.VGG import VGG_16
    image_model = VGG_16(CNN_weights_file_name)
    # this is standard VGG 16 without the last two layers 
    sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
    image_model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
    return image_model

In [7]:
def get_image_features(image_file_name, CNN_weights_file_name):
    image_features = np.zeros((1, 4096))
    im = cv2.resize(cv2.imread(image_file_name), (224, 224))
    mean_pixel = [103.939, 116.779, 123.68]
    im = im.astype(np.float32, copy=False)
    for c in range(3):
        im[:, :, c] = im[:, :, c] - mean_pixel[c]

    im = im.transpose((2,0,1)) # convert the image to RGBA
    im = np.expand_dims(im, axis=0) 
    image_features[0,:] = get_image_model(CNN_weights_file_name).predict(im)[0]
    return image_features

In [8]:
def get_VQA_model(VQA_weights_file_name):
    from models.VQA.VQA import VQA_MODEL
    vqa_model = VQA_MODEL()
    vqa_model.load_weights(VQA_weights_file_name)
    vqa_model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
    return vqa_model

In [9]:
def get_question_features(question):
    word_embeddings = spacy.load('en_vectors_web_lg')
    tokens = word_embeddings(question)
    question_tensor = np.zeros((1, 30, 300))
    for j in range(len(tokens)):
            question_tensor[0,j,:] = tokens[j].vector
    return question_tensor

In [10]:
image_file_name = 'test.jpg'
question = u"What vehicle is in the picture?"

In [11]:
warnings.filterwarnings("ignore")
image_features = get_image_features(image_file_name, CNN_weights_file_name)
question_features = get_question_features(str(question))
vqa_model = get_VQA_model(VQA_weights_file_name)
y_output = vqa_model.predict([question_features, image_features])
y_sort_index = np.argsort(y_output)
labelencoder = joblib.load(label_encoder_file_name)
for label in reversed(y_sort_index[0,-5:]):
        print (str(round(y_output[0,label]*100,2)).zfill(5), "% ", labelencoder.inverse_transform(label))




Instructions for updating:
keep_dims is deprecated, use keepdims instead
44.74 %  train
33.73 %  bicycle
10.45 %  bike
02.23 %  bus
01.47 %  scooter


In [13]:
import keras,os
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

In [24]:
trdata = ImageDataGenerator()
traindata = trdata.flow_from_directory(directory="data",target_size=(224,224))
tsdata = ImageDataGenerator()
testdata = tsdata.flow_from_directory(directory="test", target_size=(224,224))

Found 41 images belonging to 1 classes.
Found 9 images belonging to 1 classes.


In [27]:
from models.VQA.VQA import VQA_MODEL
model=VQA_MODEL()

In [28]:
from keras.optimizers import Adam
opt = Adam(lr=0.001)
model.compile(optimizer=opt, loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

In [31]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=20, verbose=1, mode='auto')
hist = model.fit_generator(steps_per_epoch=100,generator=traindata, validation_data= testdata, validation_steps=10,epochs=100,callbacks=[checkpoint,early])

Epoch 1/100


Exception in thread Thread-8:
Traceback (most recent call last):
  File "C:\Users\User\AppData\Local\Programs\Python\Python36\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "C:\Users\User\AppData\Local\Programs\Python\Python36\lib\threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "c:\users\user\onedrive\desktop\thesis\vqa\lib\site-packages\keras\engine\training.py", line 606, in data_generator_task
    generator_output = next(self._generator)
  File "c:\users\user\onedrive\desktop\thesis\vqa\lib\site-packages\keras\preprocessing\image.py", line 727, in __next__
    return self.next(*args, **kwargs)
  File "c:\users\user\onedrive\desktop\thesis\vqa\lib\site-packages\keras\preprocessing\image.py", line 960, in next
    target_size=self.target_size)
  File "c:\users\user\onedrive\desktop\thesis\vqa\lib\site-packages\keras\preprocessing\image.py", line 318, in load_img
    raise ImportError('Could not import PIL.Image. '
ImportErro

ValueError: output of generator should be a tuple `(x, y, sample_weight)` or `(x, y)`. Found: None